# https://www.kaggle.com/darkside92/detailed-examination-for-house-price-top-10

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import seaborn as sb
import matplotlib.pyplot as plt
import lightgbm as lgb
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, KFold,GridSearchCV
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.preprocessing import StandardScaler,RobustScaler,LabelEncoder,PowerTransformer
from sklearn.ensemble import GradientBoostingRegressor,StackingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import seaborn as sns  
import warnings
warnings.filterwarnings("ignore")         # for beautiful plots
from scipy import stats                 # to calculate mode, skew and ANOVA, etc.
import numpy as np # linear algebra
np.set_printoptions(np.inf)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm, skew #for some statistics
plt.rcParams['figure.figsize'] = (10, 8)
# importing alll the necessary packages to use the various classification algorithms
from sklearn.linear_model import LogisticRegression  # for Logistic Regression algorithm
from sklearn.model_selection import train_test_split #to split the dataset for training and testing
from sklearn.neighbors import KNeighborsClassifier  # for K nearest neighbours
from sklearn import svm  #for Support Vector Machine (SVM) Algorithm
from sklearn import metrics #for checking the model accuracy
from sklearn.metrics import mean_squared_error #for checking the model accuracy
from sklearn.tree import DecisionTreeClassifier #for using Decision Tree Algoithm
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
np.set_printoptions(threshold=np.inf)
import xgboost as xgb
import lightgbm as lgb
import time
import pickle
import gc

In [8]:
pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 5000)
train = pd.read_csv('../house price/train.csv')
test = pd.read_csv('../house price/test.csv')
sample_sub = pd.read_csv('../house price/sample_submission.csv')

In [9]:
train = pd.read_pickle('../house price/011data.pkl')[0:1433]
test = pd.read_pickle('../house price/011data.pkl')[1433:2919]

X = train.drop(['SalePrice','SalePrice_Log1p','Id'],axis=1)
y = train.SalePrice_Log1p
test = test.drop(['SalePrice','SalePrice_Log1p','Id'],axis=1)

def overfit_reducer(df):

    overfit = []
    for i in df.columns:
        counts = df[i].value_counts()
        zeros = counts.iloc[0]
        if zeros / len(df) * 100 > 99.9:
            overfit.append(i)
    overfit = list(overfit)
    return overfit

overfitted_features = overfit_reducer(X)
print(overfitted_features)
print(X.shape)
print(test.shape)
X.drop(overfitted_features, axis=1, inplace=True)
test.drop(overfitted_features, axis=1, inplace=True)
print('X.finalshape',X.shape)
print('test.finalshape',test.shape)

['PoolArea', 'PoolQC', 'exists_pool', 'Utilities_AllPub', 'Utilities_NoSeWa', 'Condition2_PosA', 'Condition2_PosN', 'Condition2_RRAe', 'Condition2_RRAn', 'RoofMatl_ClyTile', 'RoofMatl_Membran', 'RoofMatl_Metal', 'RoofMatl_Roll', 'Exterior1st_AsphShn', 'Exterior1st_CBlock', 'Exterior1st_ImStucc', 'Exterior2nd_CBlock', 'Exterior2nd_Other', 'Heating_Floor', 'Electrical_Mix', 'MiscFeature_TenC']
(1433, 239)
(1459, 239)
X.finalshape (1433, 218)
test.finalshape (1459, 218)


## データの標準化

In [10]:
std_scaler=StandardScaler()
rbst_scaler=RobustScaler()
power_transformer=PowerTransformer()
X_std = pd.DataFrame(std_scaler.fit_transform(X))
X_rbst = pd.DataFrame(rbst_scaler.fit_transform(X))
X_pwr = pd.DataFrame(power_transformer.fit_transform(X))

test_std = pd.DataFrame(std_scaler.transform(test))
test_rbst = pd.DataFrame(rbst_scaler.transform(test))
test_pwr = pd.DataFrame(power_transformer.transform(test))

## バリデーション設計と評価指標

In [11]:
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_indexes = []
    
    kf = KFold(n_splits=4, shuffle=True, random_state=0)
    
    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y)
        tr_pred = model.predict(tr_x)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_indexes.append(va_idx)
        print('  score Train : {:.6f}' .format(np.sqrt(mean_squared_error(tr_y, tr_pred))), 
              '  score Valid : {:.6f}' .format(np.sqrt(mean_squared_error(va_y, pred)))) 
        
    # バリデーションデータに対する予測値を連結し、その後元の順番に並べなおす
    va_indexes = np.concatenate(va_indexes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_indexes)
    pred_train = pd.DataFrame(preds[order])
    
    
    # テストデータに対する予測値の平均をとる
    preds_test = pd.DataFrame(np.mean(preds_test, axis=0))
    print('Score : {:.6f}' .format(np.sqrt(mean_squared_error(train_y, pred_train))))
    return pred_train, preds_test, model

## NN

In [6]:
conda update -n base -c defaults conda

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip uninstall tensorflow
!pip uninstall keras

In [3]:
conda install tensorflow

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [4]:
conda install keras

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [6]:
import tensorflow as tf

ImportError: Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 64, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed: 指定されたモジュールが見つかりません。


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for help.

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler

np.random.seed(10)

#create Model
#define base model
def base_model():
     model = Sequential()
     model.add(Dense(20, input_dim=218, init='normal', activation='relu'))
     model.add(Dense(10, init='normal', activation='relu'))
     model.add(Dense(1, init='normal'))
     model.compile(loss='mean_squared_error', optimizer = 'adam')
     return model

seed = 7
np.random.seed(seed)

keras_label = X_pwr.as_matrix()
NN = KerasRegressor(build_fn=base_model, nb_epoch=1000, batch_size=5,verbose=0)

NN_pred_train, NN_preds_test, NN = predict_cv(NN,  X_pwr, y, test_pwr)

ModuleNotFoundError: No module named 'keras.models'

## LightGBM
特徴量選択をしない場合（230個程度）の方が精度が高い

In [17]:
lgbm = lgb.LGBMRegressor(objective='regression',
                          num_leaves=2, 
                          learning_rate=0.035,
                          n_estimators=2177, 
                          max_bin=50, 
                          bagging_fraction=0.65,
                          bagging_freq=5, 
                          bagging_seed=7, 
                          feature_fraction=0.201, 
                          feature_fraction_seed=7,
                          n_jobs=-1)

lgbm_pred_train, lgbm_preds_test, lgbm = predict_cv(lgbm,  X_pwr, y, test_pwr)

  score Train : 0.093922   score Valid : 0.117419
  score Train : 0.092600   score Valid : 0.115492
  score Train : 0.095788   score Valid : 0.108656
  score Train : 0.093796   score Valid : 0.115409
Score : 0.114295


## Ridge回帰

In [18]:
ridge = Ridge(alpha=0.01)
ridge_pred_train, ridge_preds_test, ridge = predict_cv(ridge,  X_pwr, y, test_pwr)

  score Train : 0.088730   score Valid : 0.120938
  score Train : 0.086708   score Valid : 0.120095
  score Train : 0.090070   score Valid : 0.110762
  score Train : 0.083613   score Valid : 0.128199
Score : 0.120159


## Lasso回帰

In [19]:
lasso = Lasso(alpha=1e-8)
lasso_pred_train, lasso_preds_test, lasso = predict_cv(lasso,  X_pwr, y, test_pwr)

  score Train : 0.088741   score Valid : 0.121058
  score Train : 0.086730   score Valid : 0.119389
  score Train : 0.090123   score Valid : 0.110600
  score Train : 0.083656   score Valid : 0.128114
Score : 0.119953


## ElasticNet

In [20]:
elastic = ElasticNet(alpha=1e-8, l1_ratio=0.5)
elastic_pred_train, elastic_preds_test, elastic = predict_cv(elastic,  X_pwr, y, test_pwr)

  score Train : 0.088741   score Valid : 0.121058
  score Train : 0.086730   score Valid : 0.119389
  score Train : 0.090123   score Valid : 0.110600
  score Train : 0.083656   score Valid : 0.128114
Score : 0.119953


In [21]:
print('Count columns of test: ',len(test.columns))
print('Count columns of X: ',len(X.columns))

Count columns of test:  218
Count columns of X:  218


## trainの予測値を見る

In [22]:
lgbm_pred_train.columns = ['SalePrice']
elastic_pred_train.columns = ['SalePrice']
ridge_pred_train.columns = ['SalePrice']
lasso_pred_train.columns = ['SalePrice']

In [23]:
print('  score Train : {:.6f}' .format(np.sqrt(mean_squared_error(y, lgbm_pred_train))))
print('  score Train : {:.6f}' .format(np.sqrt(mean_squared_error(y, elastic_pred_train))))
print('  score Train : {:.6f}' .format(np.sqrt(mean_squared_error(y, ridge_pred_train))))
print('  score Train : {:.6f}' .format(np.sqrt(mean_squared_error(y, lasso_pred_train))))

  score Train : 0.114295
  score Train : 0.119953
  score Train : 0.120159
  score Train : 0.119953


In [28]:
best_pram_score = []
scores = []
for i in np.arange(0, 1, 0.1):
    for j in np.arange(0, 1-i, 0.1):
        for k in np.arange(0, 1-(i+j), 0.1):
            l = 1-(i+j+k)
            train_pred = i*lgbm_pred_train + j*elastic_pred_train + k*ridge_pred_train + l*lasso_pred_train
            score = np.sqrt(mean_squared_error(y, train_pred))
            best_pram_score.append([score, train_pred, i,j,k,l])

In [39]:
best_pram_score = pd.DataFrame(best_pram_score)
best_pram_score[best_pram_score[0] == best_pram_score[0].min()]

,0,1,2,3,4,5
202,0.109202,SalePrice 0 12.235177 1 12.15937...,0.6,0.0,0.0,0.4


## testの予測値を見る

In [51]:
lgbm_preds_test.columns = ['SalePrice']
elastic_preds_test.columns = ['SalePrice']
ridge_preds_test.columns = ['SalePrice']
lasso_preds_test.columns = ['SalePrice']

lgbm_preds_test['SalePrice'] = np.floor(np.expm1(lgbm_preds_test['SalePrice']))
elastic_preds_test['SalePrice'] = np.floor(np.expm1(elastic_preds_test['SalePrice']))
ridge_preds_test['SalePrice'] = np.floor(np.expm1(ridge_preds_test['SalePrice']))
lasso_preds_test['SalePrice'] = np.floor(np.expm1(lasso_preds_test['SalePrice']))

In [52]:
lgbm_preds_test.head()

,SalePrice
0,130672.0
1,154680.0
2,188712.0
3,200726.0
4,187596.0


In [53]:
elastic_preds_test.head()

,SalePrice
0,123674.0
1,165812.0
2,183162.0
3,201538.0
4,194563.0


In [54]:
ridge_preds_test.head()

,SalePrice
0,123344.0
1,165494.0
2,183517.0
3,201803.0
4,194415.0


In [55]:
lasso_preds_test.head()

,SalePrice
0,123674.0
1,165812.0
2,183162.0
3,201538.0
4,194563.0


## 加重平均

In [56]:
final_pred = (lasso_preds_test*0.4)+(lgbm_preds_test*0.6)
final_pred.head()

,SalePrice
0,127872.8
1,159132.8
2,186492.0
3,201050.8
4,190382.8


In [57]:
sample_sub['SalePrice'] = final_pred

In [58]:
sample_sub

,Id,SalePrice
0,1461,127872.8
1,1462,159132.8
2,1463,186492.0
3,1464,201050.8
4,1465,190382.8
5,1466,173230.0
6,1467,177910.6
7,1468,163769.6
8,1469,185655.4
9,1470,120180.4


In [60]:
sample_sub.to_csv('submission.csv',index=False)
#0.12519

 0.12414 ENet*0.5 LGBM*0.5 cols=230
 
 0.13137 Enet Only cols=230
 
 0.12423 LGBM Only cols=230
 
 0.13972 Enet Only cols=50 submissionに対しては過学習か 
 
 0.12772 ENet*0.5 LGBM*0.5 cols=50, 230
 
 0.12465 ENet*0.2 LGBM*0.8 cols=50, 230
 
